In [1]:
from IPython.display import display

from io import StringIO
import itertools
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd

from mosek.fusion import Model, Domain, Expr, ObjectiveSense, SolutionStatus


In [2]:
supply = pd.read_csv('20201007_da_co.processed.csv').set_index('id')
nsupply = len(supply)
supply.loc[supply['offer ($/MW)'] <= 0., 'capacity (MW)'] /= 1.
print(supply.shape)
supply.head()

(3617, 3)


,node,capacity (MW),offer ($/MW)
id,,,
A3153_0,North,1.0,8.16
A3153_1,North,245.0,9.54
A3153_2,North,1.0,14.27
A3153_3,North,72.0,14.28
A3165_0,Central,180.0,10.77


In [3]:
nodes = list(supply['node'].unique())
nnodes = len(nodes)
print(nodes)

['North', 'Central', 'South']


In [4]:
demand = pd.read_csv('20201007_bids_cb.processed.csv').set_index('id')
ndemand = len(demand)
assert demand.node.isin(nodes).all()
print(demand.shape)
demand.head()

(309, 3)


,node,demand (MW),bid ($/MW)
id,,,
122073561,North,6.0,2000.0
767746013,North,14.0,2000.0
122073033,Central,1228.0,2000.0
122073042,North,1644.0,2000.0
450523929,Central,132.4,2000.0


In [5]:
demand.groupby(demand['bid ($/MW)'] >= 2000)['node'].count()

bid ($/MW)
False     41
True     268
Name: node, dtype: int64

In [6]:
lines = pd.DataFrame([
    ['%s-%s' % (src, dest), src, dest, 5e3, 1e5] 
    for src, dest in itertools.combinations(nodes, 2)
], columns=['id', 'source', 'dest', 'capacity (MW)', 'susceptance (S)']).set_index('id')
nlines = len(lines)
lines

,source,dest,capacity (MW),susceptance (S)
id,,,,
North-Central,North,Central,5000.0,100000.0
North-South,North,South,5000.0,100000.0
Central-South,Central,South,5000.0,100000.0


In [7]:
pd.concat([demand.groupby('node')[['demand (MW)']].sum(),
           supply.groupby('node')[['capacity (MW)']].sum()], axis='columns')

,demand (MW),capacity (MW)
node,,
Central,28181.2,85287.0
North,12854.2,52353.8
South,16425.5,47877.5


In [8]:
M = Model('power')

log = StringIO()
M.setLogHandler(log)
theta = M.variable('theta', nnodes, Domain.inRange(
    [0.] + [-np.pi] * (nnodes - 1), 
    [0.] + [np.pi] * (nnodes - 1), 
))
pD = M.variable('pD', ndemand, Domain.inRange(0., demand['demand (MW)'].values))
pG = M.variable('pG', nsupply, Domain.inRange(0., supply['capacity (MW)'].values))
lambdaD = demand['bid ($/MW)'].values
lambdaG = supply['offer ($/MW)'].values

line_constraints = Domain.inRange(-lines['capacity (MW)'].values, 
                                  lines['capacity (MW)'].values)
line_flows = M.variable('flows', nlines, line_constraints)
src_idx = [nodes.index(x) for x in lines.source]
dst_idx = [nodes.index(x) for x in lines.dest]
phase_diff = Expr.sub(theta.pick(src_idx), theta.pick(dst_idx))
phase_flow = M.constraint(Expr.sub(line_flows, 
                                  Expr.mulElm(lines['susceptance (S)'].values, phase_diff)),
                         Domain.equalsTo(0.))
node_balance_eqs = []
for node in nodes:
    supply_idx = np.flatnonzero(supply.node == node).astype('int32')
    node_supply = Expr.sum(pG.pick(supply_idx))
    demand_idx = np.flatnonzero(demand.node == node).astype('int32')
    node_demand = Expr.sum(pD.pick(demand_idx))
    balance = Expr.sub(node_demand, node_supply)
    direction = [1 if line.source == node else 
                 -1 if line.dest == node else 
                 0 for line_idx, line in lines.iterrows()]
    balance = Expr.add(balance, Expr.dot(direction, line_flows))
    balance_eq = M.constraint(node + 'balance', balance, Domain.equalsTo(0.))
    node_balance_eqs.append(balance_eq)

obj = Expr.sub(Expr.dot(lambdaD, pD), Expr.dot(lambdaG, pG))
M.objective('obj', ObjectiveSense.Maximize, obj)
M.solve()

if M.getPrimalSolutionStatus() != SolutionStatus.Optimal:
    M.writeTask('proj1.opf')

demand['consumed (MW)'] = pD.level()
supply['supplied (MW)'] = pG.level()
lines['flow (MW)'] = line_flows.level()
buses = pd.DataFrame({
    'volt. angle (rad)': theta.level(),
    'node price ($/MW)': [e.dual()[0] for e in node_balance_eqs]
}, index=nodes)

In [9]:
summary = pd.concat([
    buses['node price ($/MW)'],
    demand.groupby('node')['consumed (MW)'].sum(),
    supply.groupby('node')['supplied (MW)'].sum(),
], axis=1)
summary.columns = ['price ($/MW)', 'consumed (MW)', 'supplied (MW)']
display(summary)
print(summary.to_latex())

,price ($/MW),consumed (MW),supplied (MW)
North,-0.00,12854.2,22854.2
Central,11.84,28178.1,23178.1
South,11.32,16367.5,11367.5


\begin{tabular}{lrrr}
\toprule
{} &  price (\$/MW) &  consumed (MW) &  supplied (MW) \\
\midrule
North   &         -0.00 &        12854.2 &        22854.2 \\
Central &         11.84 &        28178.1 &        23178.1 \\
South   &         11.32 &        16367.5 &        11367.5 \\
\bottomrule
\end{tabular}



In [10]:
print('Energy scheduled: {:g}'.format(demand['consumed (MW)'].sum()))
print('Objective value: {:g}'.format(M.primalObjValue()))
display(lines)
display(buses)
display(demand.sort_values('bid ($/MW)'))
display(supply.sort_values('offer ($/MW)'))

Energy scheduled: 57399.8
Objective value: 1.13911e+08


,source,dest,capacity (MW),susceptance (S),flow (MW)
id,,,,,
North-Central,North,Central,5000.0,100000.0,5000.0
North-South,North,South,5000.0,100000.0,5000.0
Central-South,Central,South,5000.0,100000.0,0.0


,volt. angle (rad),node price ($/MW)
North,0.00,-0.00
Central,-0.05,11.84
South,-0.05,11.32


,node,demand (MW),bid ($/MW),consumed (MW)
id,,,,
1656071284,South,58.0,4.21,0.0
1656072147_0,South,27.0,11.33,27.0
122073523_7,Central,3.1,11.47,0.0
122073523_6,Central,3.1,11.99,3.1
122073523_5,Central,3.1,12.51,3.1
...,...,...,...,...
816860633,Central,5.0,2000.00,5.0
1601140898,Central,3.0,2000.00,3.0
1066937520,Central,30.0,2000.00,30.0


,node,capacity (MW),offer ($/MW),supplied (MW)
id,,,,
A14605_0,North,1.0,-50.0,1.0
A6994_0,North,1.0,-50.0,1.0
A7548_0,North,1.0,-50.0,1.0
A6027_0,North,1.0,-50.0,1.0
A6999_0,North,1.0,-50.0,1.0
...,...,...,...,...
A14896_1,North,69.0,999.0,0.0
A14517_1,Central,19.0,999.0,0.0
A12762_1,North,23.0,999.0,0.0


In [11]:
print(log.getvalue())

MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(3) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(4) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(5) of matrix 'A'.
Problem
  Name                   : power           
  Objective sense        : max             
  Type                   : LO (linear optimization problem)
  Constraints            : 6               
  Cones                  : 0               
  Scalar variables       : 3933            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 2
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time             

In [12]:
supply.query('`offer ($/MW)` <= 0.').groupby('node')[['capacity (MW)', 'supplied (MW)']].sum()

,capacity (MW),supplied (MW)
node,,
Central,14683.6,14683.6
North,27189.0,22854.2
South,239.5,239.5


In [13]:
supply.query('`offer ($/MW)` > 0.').groupby('node')[['capacity (MW)', 'supplied (MW)']].sum()

,capacity (MW),supplied (MW)
node,,
Central,70603.4,8494.5
North,25164.8,0.0
South,47638.0,11128.0


In [14]:
demand.query('`bid ($/MW)` <= 0.').groupby('node')[['demand (MW)', 'consumed (MW)']].sum()

,demand (MW),consumed (MW)
node,,


In [15]:
demand.query('`bid ($/MW)` > 0.').groupby('node')[['demand (MW)', 'consumed (MW)']].sum()

,demand (MW),consumed (MW)
node,,
Central,28181.2,28178.1
North,12854.2,12854.2
South,16425.5,16367.5
